In [1]:
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
MODULE_HANDLE = 'https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet1k_b0/feature_vector/2'
IMAGE_SIZE = (224, 224)
NUM_CLASSES = 4 

In [3]:
def create_model():
    feature_extractor = hub.KerasLayer(MODULE_HANDLE,
                                       input_shape=IMAGE_SIZE + (3,),  # Input shape of images (height, width, channels)
                                       trainable=False)  # Freeze the weights of the pre-trained model
    
    model = models.Sequential([
        feature_extractor,
        layers.Dense(NUM_CLASSES, activation='softmax')  # Output layer with softmax activation for classification
    ])
    
    return model

# Create an instance of the model
model = create_model()

In [4]:
train_dir = 'C:/Users/akshi/Desktop/project2k24/plant disease prediction/training/dataset/train'
test_dir = 'C:/Users/akshi/Desktop/project2k24/plant disease prediction/training/dataset/test'
# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [5]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')


In [6]:
test_datagen = ImageDataGenerator(rescale=1./255)

In [7]:
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMAGE_SIZE,
    batch_size=32,
    class_mode='categorical')

Found 1820 images belonging to 4 classes.


In [8]:

validation_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=IMAGE_SIZE,
    batch_size=32,
    class_mode='categorical')

Found 343 images belonging to 4 classes.


In [9]:

history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples/train_generator.batch_size,
    epochs=50,  # You can adjust the number of epochs
    validation_data=validation_generator,
    validation_steps=validation_generator.samples/validation_generator.batch_size)

# Evaluate the model
test_loss, test_accuracy = model.evaluate(validation_generator)
print('Test Accuracy:', test_accuracy)


Epoch 1/50



56/56 [==============================] - 323s 5s/step - loss: 1.0446 - accuracy: 0.5714 - val_loss: 1.0830 - val_accuracy: 0.6327
Epoch 2/50
56/56 [==============================] - 238s 4s/step - loss: 0.8464 - accuracy: 0.7049 - val_loss: 1.0568 - val_accuracy: 0.6181
Epoch 3/50
56/56 [==============================] - 230s 4s/step - loss: 0.7581 - accuracy: 0.7407 - val_loss: 0.9590 - val_accuracy: 0.6356
Epoch 4/50
56/56 [==============================] - 271s 5s/step - loss: 0.6972 - accuracy: 0.7670 - val_loss: 0.9535 - val_accuracy: 0.6414
Epoch 5/50
56/56 [==============================] - 212s 4s/step - loss: 0.6660 - accuracy: 0.7725 - val_loss: 0.9695 - val_accuracy: 0.6239
Epoch 6/50
56/56 [==============================] - 306s 5s/step - loss: 0.6412 - accuracy: 0.7725 - val_loss: 0.9644 - val_accuracy: 0.6647
Epoch 7/50
56/56 [==============================] - 130s 2s/step - loss: 0.6125 - accuracy: 0.7874 - val_loss: 0.8320 - val_accuracy: 0.6880
Epoch 8/50
56/56 [======

In [10]:
model.save("newmodel.h5")

G:\anaconda\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [11]:
from sklearn import svm
from sklearn.metrics import classification_report, accuracy_score
import numpy as np

# Extract features using the trained CNN model
train_features = model.predict(train_generator)
train_labels = train_generator.classes

# Flatten the features
train_features_flattened = np.reshape(train_features, (train_features.shape[0], -1))

# Train an SVM model
svm_classifier = svm.SVC(kernel='linear')
svm_classifier.fit(train_features_flattened, train_labels)

# Extract features for validation data
validation_features = model.predict(validation_generator)
validation_labels = validation_generator.classes
validation_features_flattened = np.reshape(validation_features, (validation_features.shape[0], -1))

# Predict using SVM model
svm_predictions = svm_classifier.predict(validation_features_flattened)


11/11 [==============================] - 16s 1s/step


In [12]:
import joblib


joblib.dump(svm_classifier, 'svm_model1.pkl')


['svm_model1.pkl']